In [1]:
import fastf1
import fastf1.plotting
import seaborn as sns
from matplotlib import pyplot as plt
import math
import pandas as pd

In [2]:
drivers = ["ALB", "ALO", 
            "BOT", "RIC", 
            "GAS", "HAM", 
            "HUL", "LEC", 
            "MAG", "NOR", 
            "OCO", "PER", 
            "PIA", "RUS", 
            "SAI", "SAR", 
            "STR", "TSU", 
            "VER", "ZHO"]

In [3]:
session = fastf1.get_session(2023, "hungary", 'R')
session.load()

req         WARNING 	

DEFAULT CACHE ENABLED!
	Cache directory: /Users/gakroppongi/Library/Caches/fastf1.
	Size: 2.07 GB


core           INFO 	Loading data for Hungarian Grand Prix - Race [v3.0.7]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '11', '44', '81', '63', '16', '55', '14', '18', '23', '77', '3', '27', '22', '24', '20', '2', '31', '10']


In [26]:
def summerize_lap_time_stat(session, drivers):
    for driver in drivers:
        driver_laps = session.laps.pick_driver(driver).pick_quicklaps().reset_index()
        grouped_laps = driver_laps.groupby("Stint")
        mean_lap_time = grouped_laps.LapTime.mean().dt.total_seconds()
        gap_to_avg = [[None for i in range(3)] for j in range(len(driver_laps))]

        for i in range(len(driver_laps)):
            stint = driver_laps.iloc[i].Stint
            avg = mean_lap_time.iloc[int(stint-1.0)]
            lap_time = driver_laps.iloc[i].LapTime.total_seconds()
            gap_to_avg[i][0] = lap_time - avg
            gap_to_avg[i][1] = stint
            gap_to_avg[i][2] = driver_laps.iloc[i].Compound

        df = pd.DataFrame(gap_to_avg, columns=["gap", "Stint", "Compound"])
        df = df.groupby("Stint")
        df.describe()

        grouped = session.laps.pick_driver("VER").pick_quicklaps().groupby("Stint")
        grouped = grouped.LapTime
        grouped.describe()

In [25]:
def plot_box_wisker_plot_of_lap_time_to_avg(session, drivers):
    for driver in drivers:
        driver_laps = session.laps.pick_driver(driver).pick_quicklaps().reset_index()
        grouped_laps = driver_laps.groupby("Stint")
        mean_lap_time = grouped_laps.LapTime.mean().dt.total_seconds()
        gap_to_avg = [[None for i in range(3)] for j in range(len(driver_laps))]

        for i in range(len(driver_laps)):
            stint = driver_laps.iloc[i].Stint
            avg = mean_lap_time.iloc[int(stint-1.0)]
            lap_time = driver_laps.iloc[i].LapTime.total_seconds()
            gap_to_avg[i][0] = lap_time - avg
            gap_to_avg[i][1] = stint
            gap_to_avg[i][2] = driver_laps.iloc[i].Compound

        df = pd.DataFrame(gap_to_avg, columns=["gap", "Stint", "Compound"])

        # Create a figure with two subplots: one for the plot and one for the table
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

        # Create the boxplot on the first subplot
        sns.boxplot(y="gap", x="Stint", data=df, hue="Compound", palette=fastf1.plotting.COMPOUND_COLORS)

        plt.show()